## Convert xlsx data to csv format 

In [11]:
# import xlrd
# import csv

# def csv_from_excel(path, sheet, csv_name):
#     wb = xlrd.open_workbook(path)
#     sh = wb.sheet_by_name(sheet)
#     csv_file = open(csv_name, 'w')
#     wr = csv.writer(csv_file, quoting=csv.QUOTE_ALL)

#     for rownum in range(sh.nrows):
#         if sh.row_values(rownum)[0] == '':
#             return
#         else:
#             wr.writerow(sh.row_values(rownum))
#     csv_file.close()

In [12]:
# csv_from_excel("./mergeddata.xlsx", "Sheet1", "mergeddata.csv")

In [13]:
# import pandas as pd 
# import numpy as np 

# data_raw = pd.read_csv("mergeddata.csv")
# data_raw

In [14]:
# shuffle order
# data = data_raw.sample(frac=1).reset_index(drop=True)

a full pariswise construction of the current data will have 3540C2 = 6264030 entries (too much!)

## Experiment with pairwise comparison data

In [15]:
# exp_data = data[:10]
# exp_data

In [16]:
# columns = ["Price","Bedrooms","Bathrooms","Size","Crime","Transit","Shopping","Health","Entertainment","Community","Diversity","Schools","Employment"]
# columns1 = []
# columns2 = []

# for col in columns:
#     columns1.append(col+'1')
#     columns2.append(col+'2')
    
# print(columns1)
# print(columns2)

# combined_columns = columns1[1:] + columns2[1:]
# combined_columns.append("Rank")
# print(combined_columns)
# print(len(combined_columns))

In [17]:
# mat = exp_data.as_matrix()
# print(mat.shape)

In [18]:
# from scipy.special import comb
# comb(10,2)

In [19]:
# mat = np.around(mat, decimals=2)
# print(mat[0])
# print(mat[1])
# print(mat[2])
# comb_test = np.asarray([int(1 if mat[0,0] > mat[1,0] else 0)])
# print(comb_test)
# print(mat[0,1:].shape)
# print(comb_test.shape)
# con = np.concatenate((mat[0, 1:], mat[1, 1:], mat[2, 1:], comb_test))
# print(con)

In [20]:
# count = 0
# for i in range(10):
#     for j in range(i):
#         print(i,j)
#         count += 1
# print(count)

In [21]:
# new_mat = np.zeros(shape=(int(comb(mat.shape[0], 2)), mat.shape[1]*2-1))
# print(new_mat.shape)
# print(mat.shape)

# count = 0 
# for i in range(exp_data.shape[0]):
#     for j in range(i):
#         # calculate rank as entry 1 price > entry 2 price ? 1: 0
#         new_mat[count] = np.concatenate((mat[i,1:], mat[j,1:], np.asarray([1 if mat[i,0] > mat[j,0] else 0])))  
#         count += 1
        
# print(new_mat[:5])
# print(new_mat[:,24])

In [22]:
# # API
# def get_partial_data_as_matrix(data, entry):
#     return data[:entry].as_matrix()

# def generate_pairwise_data(mat, col_names):
#     new_mat = np.zeros(shape=(int(comb(mat.shape[0], 2)), mat.shape[1]*2-1))
#     print(new_mat.shape)

#     count = 0 
#     for i in range(mat.shape[0]):
#         for j in range(i):
#             # calculate rank as entry 1 price > entry 2 price ? 1: 0
#             new_mat[count] = np.concatenate((mat[i,1:], mat[j,1:], np.asarray([1 if mat[i,0] > mat[j,0] else 0])))  
#             count += 1
#     return pd.DataFrame(data=new_mat, columns=col_names)

# def save_to_csv(df, path):
#     df.to_csv(path)


In [23]:
# # ATTENTION!!!!!!
# pairwise_data = get_partial_data_as_matrix(data, data.shape[0])

# pairwise_mat = generate_pairwise_data(pairwise_data)

# print(pairwise_mat)

all non-pricing features are included + a rank feature at the end 

In [24]:
# attention
# # save_to_csv("pairwise_data.csv", pairwise_mat)

In [25]:
# pairwise_data_test = get_partial_data_as_matrix(data, 200)

# pairwise_mat_test = generate_pairwise_data(pairwise_data_test, combined_columns)

# print(pairwise_mat_test)

# save_to_csv(pairwise_mat_test, "pairwise_data_test200.csv")

# Use the below 

In [26]:
# API
import numpy as np
import pandas as pd 
from scipy.special import comb

def get_partial_data_as_matrix(data, entry, shuffle=True):
    if shuffle:
        temp = data.sample(frac=1).reset_index(drop=True)
    else:
        temp = data
    return temp[:entry], list(data.columns.values) 

def generate_euclidean_data(data):
    mat = data.as_matrix() # convert dataframe into np matrix
#     print(mat.shape)  
    new_mat = np.zeros(shape=(int(comb(mat.shape[0], 2)), mat.shape[1]*2-1))
#     print(new_mat.shape)

    count = 0 
    for i in range(mat.shape[0]):
        for j in range(i):
            # calculate rank as entry 1 price > entry 2 price ? 1: 0
            # new_mat[count] = np.concatenate((mat[i,1:], mat[j,1:], np.asarray([1 if mat[i,0] > mat[j,0] else 0]))) 
            new_mat[count] = np.concatenate((mat[i,1:], mat[j,1:], np.asarray([1 if mat[i,0] > mat[j,0] else 0])))
            count += 1
    return new_mat

def get_column_names(raw_col_names):
    columns1 = []
    columns2 = []

    for col in raw_col_names[1:]:
        columns1.append(col+'1')
        columns2.append(col+'2')

    combined_columns = columns1 + columns2
    combined_columns.append("Rank")
    return combined_columns

def normalize_data(mat):
    max_values = np.amax(mat, axis=0)
    all_zeros = not max_values.any()
    if all_zeros:
        max_shape = max_values.shape
        max_values = np.ones(shape=max_shape)
    return mat/max_values

def convert_to_dataframe(mat, col_names):
    return pd.DataFrame(data=mat, columns=col_names)    

def save_to_csv(df, path):
    df.to_csv(path)
    
def generated_euclidean_dataframe(data, entry, path, shuffle=True):
    filtered_data = data.fillna(0)
    temp, col_names = get_partial_data_as_matrix(filtered_data, entry, shuffle=shuffle)
    combined_columns = get_column_names(col_names)
    mat = generate_euclidean_data(temp)
    normalized_mat = normalize_data(mat)
    normalized_df = convert_to_dataframe(normalized_mat, combined_columns)
    save_to_csv(normalized_df, path)
    return normalized_df

# Example

In [27]:
raw_data = pd.read_csv('mergeddata_june22.csv')
# print(raw_data[:10])

In [28]:
# arguments are:  raw_data as a dataframe,  number of entries to select from raw_data, path name to the file saved
test_df = generated_euclidean_dataframe(raw_data, 10, 'pairwise_data10.csv')
test_df

,Bedrooms1,Bathrooms1,Size1,Crime1,Transit1,Shopping1,Health1,Entertainment1,Community1,Diversity1,...,size*crime2,size*transit2,size*health2,size*shopping2,size*entertainment2,size*community2,size*diversity2,size*school2,size*employment2,Rank
0,0.4,1.0,0.79980,0.008495,0.500555,0.837014,0.213996,0.758549,1.000000,0.982097,...,1.000000,0.571701,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0
1,0.2,0.5,0.69970,0.718447,0.968923,1.000000,0.185598,1.000000,0.607505,0.008951,...,1.000000,0.571701,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0
2,0.2,0.5,0.69970,0.718447,0.968923,1.000000,0.185598,1.000000,0.607505,0.008951,...,0.007845,0.500555,0.120906,0.393864,0.419447,0.638061,0.790860,0.237771,0.398063,0.0
3,0.4,0.5,0.69970,0.016990,0.038846,0.236593,0.178499,0.372021,0.021298,0.783887,...,1.000000,0.571701,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0
4,0.4,0.5,0.69970,0.016990,0.038846,0.236593,0.178499,0.372021,0.021298,0.783887,...,0.007845,0.500555,0.120906,0.393864,0.419447,0.638061,0.790860,0.237771,0.398063,0.0
5,0.4,0.5,0.69970,0.016990,0.038846,0.236593,0.178499,0.372021,0.021298,0.783887,...,0.580430,0.847656,0.091738,0.411665,0.483753,0.339111,0.006306,0.333468,0.171649,1.0
6,1.0,1.0,0.79980,0.076456,1.000000,0.947424,0.464503,0.430052,0.542596,0.882353,...,1.000000,0.571701,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0
7,1.0,1.0,0.79980,0.076456,1.000000,0.947424,0.464503,0.430052,0.542596,0.882353,...,0.007845,0.500555,0.120906,0.393864,0.419447,0.638061,0.790860,0.237771,0.398063,0.0
8,1.0,1.0,0.79980,0.076456,1.000000,0.947424,0.464503,0.430052,0.542596,0.882353,...,0.580430,0.847656,0.091738,0.411665,0.483753,0.339111,0.006306,0.333468,0.171649,1.0
9,1.0,1.0,0.79980,0.076456,1.000000,0.947424,0.464503,0.430052,0.542596,0.882353,...,0.013726,0.033984,0.088228,0.097397,0.179966,0.011889,0.552242,0.093889,0.228865,1.0


## Experiment with list-wise data